In [1]:

import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction

#Sentence tokenization
from nltk.tokenize import sent_tokenize

#Word tokenization
from nltk.tokenize import TreebankWordTokenizer

#Stemming
from nltk.stem.lancaster import LancasterStemmer

In [2]:

# Imports
# Basics
from __future__ import print_function, division
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# gensim
import gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics.pairwise as smp

from sklearn.decomposition import NMF

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
dl_df = pd.read_csv('disneyland.csv')

In [4]:
dl_df.head()

,Unnamed: 0,dates,members,reviews,titles
0,0,Reviewed 2 days ago,"Kelsie GArmstrong, Canada10",This was my 5th time visiting Disneyland Park ...,My Favourite
1,1,Reviewed 2 days ago,"SightCeeROceanside, California5213",My family and I had an amazing time. It is rat...,Expensive but a bucket list check!
2,2,Reviewed 2 days ago,Janell G21,We had a great time here. The weather was cool...,We live Disneyland
3,3,Reviewed 3 days ago,"CanadianShanEdmonton, AB69","Castmembers are friendly and helpful, most of ...","Go early, prepare your wallet, amazing experience"
4,4,Reviewed 3 days ago,"D GSan Rafael, California189","Everything is top notch, fun as always. Tip fo...",Disney Does it Right!


In [5]:
dl_df.shape

(20390, 5)

In [6]:
dca_df = pd.read_csv('dca.csv')

In [7]:
dca_df

,Unnamed: 0,dates,members,reviews,titles
0,0,Reviewed 5 days ago,Heather B93,"There were scheduled ride closures, but while ...",So many closures
1,1,Reviewed 5 days ago,"jmegg3Omaha, Nebraska3526",We visited the park two days ago and to be hon...,Not what we expected
2,2,Reviewed 5 days ago,"daveb186London, Ontario, Canada7918","We had a great time, don’t miss the nighttime ...","Excellent, even with some construction going on."
3,3,Reviewed 5 days ago,"Dawn HPotters Bar, United Kingdom63",We visited Disney California Adventure park la...,CARS LAND...WOW
4,4,Reviewed 6 days ago,"Kris BVancouver Island, Canada12",I really loved Disneyland California Adventure...,Loved Disneyland California Adventure Park
5,5,Reviewed 6 days ago,"Tara HVacaville, California14927",I could not stop taking pictures. My husband a...,My favorite part of both parks. I could have s...
6,6,Reviewed 1 week ago,"youngSaudiTravellerCardiff, United Kingdom18698",We couldn't go on all the rides as some were c...,Enjoyed
7,7,Reviewed 1 week ago,"HughlongdenTaupo, New Zealand81",Took my mum for her 50th and we both absolutel...,Worth every minute of the long plane ride
8,8,Reviewed 1 week ago,truekiwiNewZealandNew Zealand6133,We enjoyed a much better day at this park than...,Better than Disneyland!!
9,9,Reviewed 1 week ago,Canada3219624,It was surprising and disappointing that so ma...,So much closed down


In [8]:
dca_df.shape

(10990, 5)

In [9]:
#Load stopwords
stopwords = nltk.corpus.stopwords.words('english')

### Combining Disneyland and DCA Reviews into a List

Disneyland Reviews

In [10]:
dl_reviews = list(dl_df['reviews'])

In [11]:
dl_reviews

["This was my 5th time visiting Disneyland Park in Anaheim. It's my favourite over Disneyworld in Florida because everything is so close. You can walk basically anywhere saving you time and money. I can't wait to come back many more times!",
 'My family and I had an amazing time. It is rather expensive but it is a must. You forget the $ stacking up when you see your kids smiling. Get fast passes but be mindful you can only ride it once on them per ride. Spend a little more and get the hopper pass, too much to see in one day.',
 'We had a great time here. The weather was cool and lines weren’t too long. It’s a great time to come and visit.',
 'Castmembers are friendly and helpful, most of the rides are interesting and fun but food and merchandise is very expensive. I advise going at park opening or earlier as security and main gates while take at least 20 mins. By mid afternoon it’s literally a sea of people, shoulder to shoulder so it can be tough to get around and most rides have 30 m

In [12]:
dca_reviews = list(dca_df['reviews'])

In [13]:
dca_reviews

['There were scheduled ride closures, but while there, we experienced rice malfunctions on 3 separate rides and the Frozen show was cancelled unexpectedly. With our fastpass sitting all day to get on the cars ride in the late afternoon, we spent a lot of time in lines. After so many issues, I ended up counceling my 8 year old on how we need to just roll with things out of our control. However, since we paid over $400 for the day for our family, the number of these issues was really frustrating. I expect better from Disney. My kids wanted to go back to Disneyland and leave CA adventure. Soarin was definitely a highlight of the day and worth the hour + wait.',
 'We visited the park two days ago and to be honest, we were very disappointed! This review is being made by three adults who went to the park instead of having kids with us so I know that could make a difference with this particular park. However, that being said, I have been to Disneyworld one time without kids and it is great fu

In [14]:
#Combine all reviews into one list
reviews = []
for review in dl_reviews:
    reviews.append(review)

In [15]:
reviews

["This was my 5th time visiting Disneyland Park in Anaheim. It's my favourite over Disneyworld in Florida because everything is so close. You can walk basically anywhere saving you time and money. I can't wait to come back many more times!",
 'My family and I had an amazing time. It is rather expensive but it is a must. You forget the $ stacking up when you see your kids smiling. Get fast passes but be mindful you can only ride it once on them per ride. Spend a little more and get the hopper pass, too much to see in one day.',
 'We had a great time here. The weather was cool and lines weren’t too long. It’s a great time to come and visit.',
 'Castmembers are friendly and helpful, most of the rides are interesting and fun but food and merchandise is very expensive. I advise going at park opening or earlier as security and main gates while take at least 20 mins. By mid afternoon it’s literally a sea of people, shoulder to shoulder so it can be tough to get around and most rides have 30 m

In [16]:
len(reviews)

20390

In [17]:
for review in dca_reviews:
    reviews.append(review)

In [18]:
len(reviews)

31380

In [19]:
len(set(reviews))

29621

In [20]:
#Create a list without duplicates
unique_reviews = list(set(reviews))


# TFIDF

2-4 ngrams

In [21]:
tfidf = TfidfVectorizer(stop_words="english", 
                        token_pattern="\\b[a-zA-Z][a-zA-Z]+\\b", #words with >= 2 alpha chars 
                        ngram_range=(2,4),
                        min_df=10)
tfidf_vecs = tfidf.fit_transform(unique_reviews)
pd.DataFrame(tfidf_vecs.todense(), 
             columns=tfidf.get_feature_names()
            ).head()

,able advantage,able afford,able attractions,able bodied,able bring,able california,able catch,able characters,able come,able day,...,youngest son,yr old,yr old daughter,yr old son,yr olds,yrs ago,yrs old,yummy food,zone tower,zone tower terror
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Convert sparse matrix of counts to a gensim corpus
# Need to transpose it for gensim which wants 
# terms by docs instead of docs by terms
tfidf_corpus = matutils.Sparse2Corpus(tfidf_vecs.transpose())

# Row indices
id2word = dict((v, k) for k, v in tfidf.vocabulary_.items())

# This is a hack for Python 3!
id2word = corpora.Dictionary.from_corpus(tfidf_corpus, 
                                         id2word=id2word)

2018-03-09 02:48:27,027 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-09 02:48:27,227 : INFO : adding document #10000 to Dictionary(0 unique tokens: [])
2018-03-09 02:48:27,429 : INFO : adding document #20000 to Dictionary(0 unique tokens: [])
2018-03-09 02:48:27,706 : INFO : built Dictionary(19444 unique tokens: ['park great', 'great younger', 'younger kids', 'pixar movies', 'girls loved']...) from 29621 documents (total 120174 corpus positions)


# NMF with 20 topics

In [23]:
from sklearn.decomposition import NMF

Adjust number of components

In [24]:
nmf = NMF(n_components=20, random_state=1)
nmf_vecs = nmf.fit_transform(tfidf_vecs)

## Try

In [25]:
vocab = np.array(tfidf.get_feature_names())

In [26]:
num_top_words = 20

In [27]:
len(vocab)

19444

In [33]:
topic_words = []

In [34]:
for topic in nmf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])

In [35]:
doctopic = nmf.fit_transform(tfidf_vecs)

In [37]:
new_doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [38]:
new_doctopic

array([[0.        , 0.00189802, 0.        , ..., 0.        , 0.02215845,
        0.05213252],
       [0.        , 0.        , 0.        , ..., 0.01651132, 0.0159438 ,
        0.02177799],
       [0.05593944, 0.01413438, 0.0416661 , ..., 0.01990714, 0.00633993,
        0.11752272],
       ...,
       [0.00461606, 0.        , 0.        , ..., 0.        , 0.0392464 ,
        0.02119013],
       [0.00268504, 0.        , 0.        , ..., 0.00391117, 0.        ,
        0.05370911],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [39]:
len(topic_words)

20

In [40]:
topic_words

[['space mountain',
  'indiana jones',
  'splash mountain',
  'haunted mansion',
  'pirates caribbean',
  'small world',
  'thunder mountain',
  'star tours',
  'star wars',
  'big thunder',
  'jones ride',
  'indiana jones ride',
  'main street',
  'big thunder mountain',
  'mountain indiana',
  'mountain indiana jones',
  'peter pan',
  'mountain railroad',
  'thunder mountain railroad',
  'jungle cruise'],
 ['place earth',
  'happiest place',
  'happiest place earth',
  'truly happiest place',
  'truly happiest',
  'truly happiest place earth',
  'disneyland happiest',
  'disneyland happiest place',
  'disneyland happiest place earth',
  'really happiest place',
  'really happiest',
  'really happiest place earth',
  'say happiest place',
  'say happiest',
  'say happiest place earth',
  'definitely happiest place',
  'definitely happiest',
  'definitely happiest place earth',
  'disneyland really happiest',
  'disneyland really happiest place'],
 ['california adventure',
  'disneyl

In [41]:
pd.DataFrame(new_doctopic)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000000,0.001898,0.000000,0.000332,0.175354,0.009447,0.438345,0.000000,0.018203,0.036547,0.010851,0.047749,0.000000,0.002709,0.058683,0.079124,0.046468,0.000000,0.022158,0.052133
1,0.000000,0.000000,0.000000,0.000244,0.000000,0.370782,0.000000,0.001981,0.022287,0.000000,0.010777,0.000000,0.539695,0.000000,0.000000,0.000000,0.000000,0.016511,0.015944,0.021778
2,0.055939,0.014134,0.041666,0.017074,0.000913,0.020714,0.000000,0.041858,0.070378,0.010561,0.057839,0.000000,0.050036,0.000000,0.102783,0.011543,0.360791,0.019907,0.006340,0.117523
3,0.000000,0.000891,0.119282,0.000000,0.000000,0.000000,0.004697,0.525989,0.015886,0.000000,0.005816,0.000000,0.004678,0.000000,0.000000,0.000000,0.000000,0.000000,0.287178,0.035582
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.460772,0.000000,0.000000,0.000000,0.000000,0.000000,0.539228,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.409822,0.000000,0.001915,0.003719,0.000000,0.000000,0.577933,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006610,0.000000
6,0.000000,0.000000,0.489150,0.510850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.465930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.534070,0.000000,0.000000,0.000000
8,0.002379,0.000000,0.045622,0.001123,0.000000,0.012883,0.038877,0.025945,0.100621,0.106720,0.000000,0.247704,0.000000,0.015944,0.000000,0.024634,0.037060,0.000000,0.103031,0.237458
9,0.024521,0.003016,0.000000,0.002101,0.005728,0.028988,0.186392,0.022887,0.045502,0.115146,0.218435,0.237050,0.024555,0.000000,0.000000,0.000000,0.000000,0.025591,0.052577,0.007509


### Each review is some percentage of this topic

In [43]:
#topic and topic words(each column)
df = pd.DataFrame(topic_words)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,space mountain,indiana jones,splash mountain,haunted mansion,pirates caribbean,small world,thunder mountain,star tours,star wars,big thunder,jones ride,indiana jones ride,main street,big thunder mountain,mountain indiana,mountain indiana jones,peter pan,mountain railroad,thunder mountain railroad,jungle cruise
1,place earth,happiest place,happiest place earth,truly happiest place,truly happiest,truly happiest place earth,disneyland happiest,disneyland happiest place,disneyland happiest place earth,really happiest place,really happiest,really happiest place earth,say happiest place,say happiest,say happiest place earth,definitely happiest place,definitely happiest,definitely happiest place earth,disneyland really happiest,disneyland really happiest place
2,california adventure,disneyland california,disneyland california adventure,california adventure disneyland,adventure disneyland,disneyland california adventure park,adventure great,days disneyland,california adventure great,time california,time california adventure,day california adventure,day california,love california adventure,spent days,days disneyland california,adventure day,california adventure day,love california,days disneyland california adventure
3,disney world,walt disney,walt disney world,disney world times,world times,world florida,disney world florida,magic kingdom,disney world orlando,world orlando,disney land,ve disney,disneyland disney world,ve disney world,disneyland disney,smaller disney,compared disney world,smaller disney world,compared disney,world disneyland
4,radiator springs,springs racers,radiator springs racers,ride radiator springs,ride radiator,single rider,ride radiator springs racers,springs racers ride,racers ride,radiator springs racers ride,springs ride,radiator springs ride,pass radiator,pass radiator springs,fast pass radiator,fast pass radiator springs,single rider line,rider line,loved radiator springs,loved radiator
5,great time,time disneyland,great time disneyland,family great,family great time,great time family,time family,great time park,time park,kids great time,kids great,disneyland great time,disneyland great,time visit,great time kids,time kids,great time visit,overall great time,overall great,time lines
6,fast pass,use fast pass,use fast,cars ride,pass ride,fast pass ride,rides fast pass,pass line,rides fast,fast pass line,pass time,pass rides,fast pass time,fast pass rides,ride fast pass,ride fast,popular rides,make sure,single rider,pass cars
7,park hopper,day park,day park hopper,hopper pass,park hopper pass,day park hopper pass,hopper ticket,park hopper ticket,hopper tickets,park hopper tickets,day park hopper ticket,park hopper passes,bought day,bought day park,hopper passes,bought day park hopper,park day,day hopper,got day,got day park
8,year old,old daughter,year old daughter,old son,took year,took year old,year old son,old year,old year old,year old year,year old year old,year old loved,old loved,old granddaughter,year old granddaughter,disneyland year,year old grandson,old grandson,disneyland year old,time year
9,cars land,new cars,new cars land,land amazing,cars land amazing,land great,cars land great,loved cars,loved cars land,land awesome,cars land awesome,like movie,cars ride,new cars ride,park cars land,just like,park cars,bugs land,land fun,cars land ride


# NMF with 30 topics

In [44]:
from sklearn.decomposition import NMF

Adjust number of components

In [45]:
nmf = NMF(n_components=30, random_state=1)
nmf_vecs = nmf.fit_transform(tfidf_vecs)

## Try

In [46]:
vocab = np.array(tfidf.get_feature_names())

In [47]:
num_top_words = 20

In [48]:
len(vocab)

19444

In [49]:
topic_words = []

In [50]:
for topic in nmf.components_:
    word_idx = np.argsort(topic)[::-1][0:num_top_words]
    topic_words.append([vocab[i] for i in word_idx])

In [51]:
doctopic = nmf.fit_transform(tfidf_vecs)

In [52]:
new_doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [53]:
new_doctopic

array([[1.13598844e-01, 0.00000000e+00, 2.62221886e-03, ...,
        4.30201150e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.06028141e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.90654079e-02, 6.81169557e-02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.44949854e-04, 0.00000000e+00, ...,
        4.71162954e-03, 0.00000000e+00, 1.94672068e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.18286826e-01, 0.00000000e+00, 0.00000000e+00]])

In [54]:
len(topic_words)

30

In [55]:
topic_words

[['fast pass',
  'use fast pass',
  'use fast',
  'pass ride',
  'fast pass ride',
  'rides fast pass',
  'rides fast',
  'pass line',
  'fast pass line',
  'pass time',
  'fast pass time',
  'pass rides',
  'fast pass rides',
  'ride fast pass',
  'ride fast',
  'popular rides',
  'line fast pass',
  'line fast',
  'fast pass tickets',
  'pass tickets'],
 ['place earth',
  'happiest place',
  'happiest place earth',
  'disneyland happiest',
  'disneyland happiest place',
  'disneyland happiest place earth',
  'really happiest place',
  'really happiest',
  'really happiest place earth',
  'say happiest',
  'say happiest place',
  'say happiest place earth',
  'definitely happiest place',
  'definitely happiest',
  'disneyland really happiest',
  'definitely happiest place earth',
  'disneyland really happiest place',
  'place earth just',
  'earth just',
  'disneyland really'],
 ['california adventure',
  'california adventure disneyland',
  'adventure disneyland',
  'time california'

In [56]:
pd.DataFrame(new_doctopic)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.113599,0.000000,0.002622,0.000000,0.133533,0.000000,0.000000,0.000000,0.006875,0.023542,...,0.268163,0.005889,0.023233,0.000000,0.000180,0.331974,0.000000,0.004302,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.001018,0.000000,0.338279,0.018336,0.003371,0.021381,0.000000,...,0.000000,0.012240,0.012288,0.017744,0.000000,0.000000,0.000000,0.000000,0.010603,0.000000
2,0.000000,0.000000,0.000000,0.010417,0.000452,0.010044,0.277006,0.018463,0.034712,0.007015,...,0.072044,0.048953,0.030853,0.006767,0.011627,0.000000,0.003898,0.000000,0.019065,0.068117
3,0.002081,0.001813,0.143859,0.000000,0.000000,0.000000,0.001346,0.515008,0.017591,0.000000,...,0.000000,0.025511,0.001153,0.004105,0.005094,0.000000,0.003571,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.455873,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.301534,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.005009,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.150005,0.420124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.002436,0.000000,0.000632,0.000000,0.000000,0.000000,0.000000,0.419755
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.460190,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004847,0.000000
8,0.007293,0.000000,0.046194,0.000442,0.000000,0.003974,0.000000,0.012742,0.070591,0.070243,...,0.103599,0.099618,0.021326,0.007416,0.052482,0.200096,0.000835,0.038200,0.002348,0.004161
9,0.002432,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009045,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.841375,0.000000,0.001490,0.000000,0.000833,0.000000


### Each review is some percentage of this topic

In [57]:
#topic and topic words(each column)
df = pd.DataFrame(topic_words)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,fast pass,use fast pass,use fast,pass ride,fast pass ride,rides fast pass,rides fast,pass line,fast pass line,pass time,fast pass time,pass rides,fast pass rides,ride fast pass,ride fast,popular rides,line fast pass,line fast,fast pass tickets,pass tickets
1,place earth,happiest place,happiest place earth,disneyland happiest,disneyland happiest place,disneyland happiest place earth,really happiest place,really happiest,really happiest place earth,say happiest,say happiest place,say happiest place earth,definitely happiest place,definitely happiest,disneyland really happiest,definitely happiest place earth,disneyland really happiest place,place earth just,earth just,disneyland really
2,california adventure,california adventure disneyland,adventure disneyland,time california,time california adventure,adventure great,california adventure great,love california adventure,love california,day california adventure,day california,loved california adventure,adventure fun,california adventure fun,loved california,visit california adventure,visit california,rides california adventure,adventure day,rides california
3,disney world,walt disney,walt disney world,disney world times,world times,world florida,disney world florida,magic kingdom,disney world orlando,world orlando,disney land,ve disney,disneyland disney world,ve disney world,disneyland disney,smaller disney,compared disney world,smaller disney world,compared disney,world disneyland
4,radiator springs,springs racers,radiator springs racers,ride radiator springs,ride radiator,single rider,ride radiator springs racers,springs racers ride,radiator springs racers ride,racers ride,springs ride,radiator springs ride,pass radiator,pass radiator springs,fast pass radiator,fast pass radiator springs,single rider line,rider line,best ride,loved radiator springs
5,great time,time disneyland,great time disneyland,family great,family great time,great time family,time family,great time park,time park,kids great time,kids great,disneyland great time,great time kids,time visit,time kids,disneyland great,great time visit,overall great time,overall great,time lines
6,star wars,wars land,star wars land,new star,new star wars,night parade,paint night,paint night parade,hyperspace mountain,wars ride,star wars ride,star wars area,wars area,star tours,new star wars land,mountain star wars,mountain star,wait star wars,wait star,going disneyland
7,park hopper,day park,day park hopper,hopper pass,park hopper pass,day park hopper pass,hopper ticket,park hopper ticket,hopper tickets,park hopper tickets,day park hopper ticket,park hopper passes,bought day,bought day park,hopper passes,bought day park hopper,park day,got day,day hopper,got day park
8,year old,old daughter,year old daughter,old son,took year,took year old,year old son,old year,old year old,year old year,year old year old,year old loved,old loved,old granddaughter,year old granddaughter,disneyland year,year old grandson,old grandson,disneyland year old,went year old
9,cars land,new cars land,new cars,land amazing,cars land amazing,land great,cars land great,loved cars land,land awesome,cars land awesome,loved cars,like movie,park cars land,park cars,just like,bugs land,land fun,cars land ride,land ride,addition cars land
